In [ ]:
import pandas as pd
import numpy as np
import os
import sys

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

import mygene
from combat.pycombat import pycombat
import scipy.stats as stats
import pickle

%load_ext autoreload
%autoreload 2

module_path = os.path.abspath(os.path.join('../src'))
sys.path.insert(0, module_path)

from utils import aggregate_df
from PABCfiles import *

In [ ]:
def readin(dfpath, columnspath, folderpath):
# returns the df generated, and the columns generated. Made it a function cause it's the same for the pabc and nonpabc folders
    print("Reading from file")
    df = pd.DataFrame()
    cols = []
    i = 1

    # for each file (sample) in the folder
    for f in os.listdir(folderpath):
        erstatus = f.split(",")[3] 
        label = f.split(",")[0]

        if os.path.isfile(os.path.join(folderpath, f)) and erstatus == " ER pos": # using only ER-positive tumours
            patientfile = os.path.join(folderpath, f)

            if i == 1: 
                # generating the new df based on the sample being read
                patientdf = pd.read_csv(patientfile, skiprows=[0,1], delimiter="\t")
                patientdf.columns = ['ID_REF', label +' VALUE']

                # setting that as the growing df dataframe 
                df = patientdf.copy() 
                cols.append(label +' VALUE')

            else: 
                # generating the new df based on the sample being read
                patientdf = pd.read_csv(patientfile, skiprows=[0,1], delimiter="\t")
                patientdf.columns = ['ID_REF', label +' VALUE']

                # merging that with the growing df
                df = df.merge(patientdf, on='ID_REF', how='inner')
                cols.append(label +' VALUE')

            i = i + 1
    
    file = open(dfpath,"wb")
    pickle.dump(df,file)
    file.close()
    
    file = open(columnspath, "wb")
    pickle.dump(cols, file)
    file.close()

    return df, cols # isn't used here but could be
    
# loading the PABC data
# platform used in this study: Affymetrix Human Genome U133 Plus 2.0 Array
if not (os.path.exists(pabcpatientsdfpath) and os.path.exists(pabcpatientcolumnspath)):
    readin(dfpath=pabcpatientsdfpath, columnspath=pabcpatientcolumnspath, folderpath=pabcpatientspath)

file = open(pabcpatientsdfpath,'rb')
pabcpatientsdf = pickle.load(file)
file.close()

file = open(pabcpatientcolumnspath,'rb')
pabcpatientscolumns = pickle.load(file)
file.close()

# loading the nonPABC data
# platform used in this study: Affymetrix Human Genome U133 Plus 2.0 Array (as above)
if not (os.path.exists(nonpabcpatientsdfpath) and os.path.exists(nonpabcpatientcolumnspath)):
    readin(dfpath=nonpabcpatientsdfpath, columnspath=nonpabcpatientcolumnspath, folderpath=nonpabcpatientspath)

file = open(nonpabcpatientsdfpath,'rb')
nonpabcpatientsdf = pickle.load(file)
file.close()

file = open(nonpabcpatientcolumnspath,'rb')
nonpabcpatientscolumns = pickle.load(file)
file.close()


In [ ]:
# AFFY HG U133 Plus 2 probe to ENSEMBL stable Gene id dictionary generated here: 
# http://www.ensembl.org/biomart/martview/5fee6e97d6077c05ef90f4d3b0b7b6b5

affy_to_ensembl = pd.read_csv(biomartaffymetrixpath, sep='\t').dropna()
affy_to_ensembl.head()
# note: there are duplicates in both the Gene stable ID column and the AFFY HG U133 Plus 2 probe column
# this analysis converts the study's probes to Gene stable IDs (ensembl IDs) so it's okay if two
# probes match to the same Gene stable ID, and it is likely this will happen. They are collapsed based on max value

In [ ]:
# gene symbol converter method that creates a dictionary from one type of gene symbol to another
# uses DAVID gene conversion table (affymetrix_DAVID) to convert from affymetrix probe ID to ensembl ID
# uses SynGO gene conversion table (ensembl_synGO) to convert from ensembl ID to HGNC

def gene_dict(df, col1, col2):
    d = dict()
    for i in range(df.shape[0]):
        if df.iloc[i][col1] not in set(d.keys()):
            d[df.iloc[i][col1]] = [df.iloc[i][col2]]
        else:
            d[df.iloc[i][col1]].append(df.iloc[i][col2])
    return d

def convert(name, ref_dict):
    if name not in set(ref_dict.keys()):
        #print("NOT MATCHED: "+ str(name))
        return 0
    return ref_dict[name][0] # return the first ID that was matched


affymetrix_to_ensembl_dict = gene_dict(affy_to_ensembl, "AFFY HG U133 Plus 2 probe", 'Gene stable ID')

In [ ]:
nonpabcpatientsdf['Gene Symbol'] = nonpabcpatientsdf.apply(lambda row: convert(row['ID_REF'], affymetrix_to_ensembl_dict), axis=1)
pabcpatientsdf['Gene Symbol'] = pabcpatientsdf.apply(lambda row: convert(row['ID_REF'], affymetrix_to_ensembl_dict), axis=1)

nonpabcpatientsdf = nonpabcpatientsdf[nonpabcpatientsdf['Gene Symbol'] != 0]
pabcpatientsdf = pabcpatientsdf[pabcpatientsdf['Gene Symbol'] != 0]


In [ ]:
class HGNCConvert:
    def __init__(self):
        self.mg = mygene.MyGeneInfo()
        self.unmatched = []

    def geneconv(self, g):
        k = self.mg.getgene(g.split('.')[0], fields = 'symbol')
        if isinstance(k, list):
            print(g)
            print(k)
            self.unmatched.append(k)
            return 0
        if k is not None and 'symbol' in list(k.keys()):
            return k['symbol']
        else: 
            self.unmatched.append(k)
            return 0
        
    def reset(self):
        self.unmatched = []


In [ ]:
# for each group of columns in the dictionary, perform each listed operation for each row
# add those resulting columns to the new dataframe

ensembl_to_HGNC = HGNCConvert()

if not (os.path.exists(tumourdfpath)):
    tumourdf = pd.merge(pabcpatientsdf, nonpabcpatientsdf, on = 'ID_REF', how='inner')
    tumourdf['Ensembl ID'] = tumourdf.apply(lambda row: convert(row['ID_REF'], affymetrix_to_ensembl_dict), axis=1)

    ensembl_to_symbol = {ens_id: ensembl_to_HGNC.geneconv(ens_id) for ens_id in tumourdf['Ensembl ID'].unique()}
    tumourdf['Gene Symbol'] = tumourdf['Ensembl ID'].map(ensembl_to_symbol)
    tumourdf = tumourdf[tumourdf['Gene Symbol'].notna()]
    tumourdf = tumourdf[tumourdf['Ensembl ID'] != 0]
    
    file = open(tumourdfpath, "wb")
    pickle.dump(tumourdf, file)
    file.close()

file = open(tumourdfpath,'rb')
tumourdf = pickle.load(file)
file.close()

In [ ]:
expression_data = tumourdf.copy().drop(columns=["gene_id", "Gene Symbol", "Geneid", 'ID_REF', 'Ensembl ID'], errors='ignore')
expression_data = expression_data[~(expression_data == 0).all(axis=1)] # drop rows with all zeros 
expression_data = expression_data[~(pd.isna(expression_data)).any(axis=1)] # drop rows with any NaN 
expression_data_T = expression_data.T


scaler = StandardScaler()
expression_scaled = scaler.fit_transform(expression_data_T)

pca = PCA(n_components=2)  
pca_result = pca.fit_transform(expression_scaled)


pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df["Sample"] = expression_data_T.index
pca_df["Group"] = pca_df["Sample"].apply(lambda x: 'PABC' if x in pabcpatientscolumns else 
                                                    'nonPABC' if x in nonpabcpatientscolumns else
                                                    'UNDEFINED')
# specific to this PABC study


plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2',
                hue="Group",  
                data=pca_df,
                palette="Set2",  
                )
 
plt.title("PABC and nonPABC transcriptomic profiles")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
def conv(g):
    k = mg.getgene(g.split('.')[0])
    if isinstance(k, list):
        print(g)
        print(k)
        return 0
    if k is not None and 'symbol' in list(k.keys()):
        return k['symbol']
    else: 
        unmatched.append(k)
        return 0

if not (os.path.exists(ourdatapath)):
    mg = mygene.MyGeneInfo()
    unmatched = []
    
    ourdata = ourdataoriginal.copy()
    ourdata['Gene Symbol'] = ourdata['Geneid'].apply(conv)

    file = open(ourdatapath,'wb')
    pickle.dump(ourdata,file)
    file.close()


file = open(ourdatapath,'rb')
ourdata = pickle.load(file)
file.close()

ourdata_grouped = ourdata.copy().groupby('Gene Symbol').max()
ourdata_grouped = ourdata_grouped.loc[~(ourdata_grouped[ourdataTPMcolumns] == 0).all(axis=1)]
tumourdf_reset = tumourdf.copy().groupby('Gene Symbol').max()
tumourdf_reset = tumourdf_reset.loc[~(tumourdf_reset[pabcpatientscolumns+nonpabcpatientscolumns] == 0).all(axis=1)]

PABCdf = pd.merge(ourdata_grouped, tumourdf_reset, how='inner', on='Gene Symbol').reset_index()

numbercols = ourdataTPMcolumns+pabcpatientscolumns+nonpabcpatientscolumns

PABCdf[numbercols] = PABCdf[numbercols].apply(lambda x: np.log2(x + 1)) 

PABCdf # old number of rows with unique gene symbols: 18978 . after using DAVID data: 19246

In [ ]:
ourdata[ourdata['Gene Symbol'].isin(k)]['D-Bewo-CT-M1'].max()
#tumourdf_reset[tumourdf_reset['Gene Symbol'].isin(j)][pabcpatientscolumns+nonpabcpatientscolumns].sum()

In [ ]:
expression_data = PABCdf.copy().drop(columns=["Ensembl ID", "Geneid", 'Gene Symbol', 'ID_REF'], errors='ignore')
expression_data = expression_data[~(expression_data == 0).all(axis=1)] # drop rows with all zeros 
expression_data = expression_data[~(pd.isna(expression_data)).any(axis=1)] # drop rows with any NaN 
expression_data_T = expression_data.T


scaler = StandardScaler()
expression_scaled = scaler.fit_transform(expression_data_T)

pca = PCA(n_components=2)  
pca_result = pca.fit_transform(expression_scaled)


pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
pca_df["Sample"] = expression_data_T.index
pca_df["Group"] = pca_df["Sample"].apply(lambda x: 'PABC' if x in pabcpatientscolumns else 
                                                    'nonPABC' if x in nonpabcpatientscolumns else
                                                    "D-MCF7" if x.split("-")[0] + x.split("-")[1] == "DMCF7" else
                                                    "S-MCF7" if x.split("-")[0] + x.split("-")[1] == "SMCF7" else
                                                    "D-BEWO" if x.split("-")[0] + x.split("-")[1] == "DBewo" else
                                                    "S-BEWO" if x.split("-")[0] + x.split("-")[1] == "SBewo" else
                                                    "UNDEFINED")
# specific to this PABC study

plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2',
                hue="Group",  
                data=pca_df,
                palette="Set2",  
                )
 
plt.title("PABC and nonPABC transcriptomic profiles")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# adapted from the pyCombat documentation:
df_expression = PABCdf.copy()[ourdataTPMcolumns + pabcpatientscolumns + nonpabcpatientscolumns + ['Gene Symbol']]

# Set index
df_expression.set_index(['Gene Symbol'], inplace=True)

# Save original index for later
original_index = df_expression.index

dataset1 = ourdata_grouped[ourdataTPMcolumns]
dataset2 = tumourdf_reset.copy()[pabcpatientscolumns + nonpabcpatientscolumns]

print(len(df_expression.columns), len(dataset1.columns), len(dataset2.columns))
print(len(df_expression.columns) == len(dataset1.columns) + len(dataset2.columns))

# we generate the list of batches
batch = []
datasets = [dataset1, dataset2]
for j in range(len(datasets)):
    batch.extend([j for _ in range(len(datasets[j].columns))])

# Drop 0-variance
zero_var_mask = df_expression.var(axis=1) == 0
df_zero_var = df_expression[zero_var_mask]
df_variable = df_expression[~zero_var_mask]

# run pyComBat
df_corrected_var = pycombat(df_variable,batch)

df_corrected = pd.concat([df_corrected_var, df_zero_var], axis=0)
df_corrected = df_corrected.loc[original_index]

df_corrected.reset_index(inplace=True)
df_corrected[df_corrected['D-Bewo-CT-M1'] == df_corrected['D-Bewo-CT-M1'].max()]


In [ ]:
PABCgenes = ['HSPA5', 'BRCA1', 'BRCA2', 'SLC23A2', 'SLC23A3', 'CS', 'CPT2', 'MVK', 'NRAS','HRAS', 'RHOA', 'ROCK2', 'TGFB1', 'CDH2']
samplesdict = {"PABC patients": pabcpatientscolumns,
            #    'nonPABC patients': nonpabcpatientscolumns,
            #    "D-MCF7-M":["D-MCF7-M1","D-MCF7-M2", "D-MCF7-M3"], 
            #    "D-MCF7-S":["D-MCF7-S1", "D-MCF7-S2", "D-MCF7-S3"], 
               "D-MCF7-CT-S":["D-MCF7-CT-S1", "D-MCF7-CT-S2", "D-MCF7-CT-S3"],
               "D-MCF7-CT-M":["D-MCF7-CT-M1", "D-MCF7-CT-M2", "D-MCF7-CT-M3"]}

functionsdict = {"mean":'mean', "STD":np.std}
columnstokeepdict = {"Gene Symbol": "Gene Symbol",
                     'p-value for PABC patients vs D-MCF7-CT-M':'p-value for PABC patients vs D-MCF7-CT-M',
                     'p-value for PABC patients vs D-MCF7-CT-S':'p-value for PABC patients vs D-MCF7-CT-S'}

def significance(row, populationcol, popmeancol):
    data = row[populationcol].astype(float).values  
    popmean = row[popmeancol].astype(float).mean()
    pvalue = stats.ttest_1samp(a=data, popmean=popmean)
    return pvalue[1]

Mphasecomparison = {"patients": pabcpatientscolumns, "cellcondition":["D-MCF7-CT-M1", "D-MCF7-CT-M2", "D-MCF7-CT-M3"]}
Sphasecomparison = {"patients": pabcpatientscolumns, "cellcondition":["D-MCF7-CT-S1", "D-MCF7-CT-S2", "D-MCF7-CT-S3"]}

PABCdflook = df_corrected[df_corrected['Gene Symbol'].isin(PABCgenes)].copy()

PABCdflook['p-value for PABC patients vs D-MCF7-CT-M'] = PABCdflook.apply(lambda row: 
                                          significance(row, Mphasecomparison["patients"], 
                                                            Mphasecomparison["cellcondition"]), 
                                                            axis=1)
PABCdflook['p-value for PABC patients vs D-MCF7-CT-S'] = PABCdflook.apply(lambda row: 
                                          significance(row, Sphasecomparison["patients"], 
                                                            Sphasecomparison["cellcondition"]), 
                                                            axis=1)

aggregate_df(PABCdflook, samplesdict, functionsdict, columnstokeepdict).set_index('Gene Symbol')

In [ ]:
columnstokeepdict = {'p-value for PABC patients vs D-MCF7-CT-M':'p-value for PABC patients vs D-MCF7-CT-M',
                     'p-value for PABC patients vs D-MCF7-CT-S':'p-value for PABC patients vs D-MCF7-CT-S',
                     }

selected = df_corrected[df_corrected['Gene Symbol'].isin(PABCgenes)]

PABCdf_calculated = selected.copy().groupby('Gene Symbol').max()

PABCdf_calculated['p-value for PABC patients vs D-MCF7-CT-M'] = PABCdf_calculated.apply(lambda row: 
                                          significance(row, Mphasecomparison["patients"], 
                                                            Mphasecomparison["cellcondition"]), 
                                                            axis=1)
PABCdf_calculated['p-value for PABC patients vs D-MCF7-CT-S'] = PABCdf_calculated.apply(lambda row: 
                                          significance(row, Sphasecomparison["patients"], 
                                                            Sphasecomparison["cellcondition"]), 
                                                            axis=1)


res = aggregate_df(PABCdf_calculated, samplesdict, functionsdict, {})


In [ ]:
data = res[res.index.isin(PABCgenes)].copy()
resultspath = ""
writer = pd.ExcelWriter(resultspath, engine='xlsxwriter')

for gene in data.index:
    df = data[data.index == gene]
    df.to_excel(writer, sheet_name=gene)

writer.close()
